### creating the memory features

In [ ]:
from groq import Client
from os import getenv
import json
# initiating the class to access the model
client = Client(
    api_key = getenv('GROQ_API_KEY')
)
# initiating the chat history
# this prompt is missing context
SYSTEM_PROMPT_CONFIG = """You are a digital consultant specializing in Australia's evolving greenhouse gas (GHG) emission regulations.
Your task is to help companies navigate the complexities of compliance, accurate emission calculations, and industry-specific scope definitions.
You also need to classify whether the user's question falls into the topics: ['legal', 'financial', 'others'] where 'others' is any topic that is
neither legal of financial.
Please only return JSON object the following structure:
{
    "response" : your answer to the user,
    "category" : one of elements of ['legal', 'financial', 'others']
}
"""
# this is how to store the memory of the chat
conversation = [
    {
        'role' : 'system',
        'content' : SYSTEM_PROMPT_CONFIG
    }
]
# sample user question
user_prompt = 'how can i sue my employer?'
# add memory to the conversation
conversation.append(
    {
        'role' : 'user',
        'content' : user_prompt
    }
)
# generating the response
response = client.chat.completions.create(
    messages = conversation,
    model = 'llama-3.3-70b-versatile',
    # lower values of temperature are related to less randomness
    temperature = 0.8,
    # maximun number of tokens to generate
    max_completion_tokens = 600,
    # A stop sequence is a predefined or user-specified text string that
    # signals an AI to stop generating content, ensuring its responses
    # remain focused and concise. Examples include punctuation marks and
    # markers like "[end]".
    stop = None,
    # If set, partial message deltas will be sent.
    stream = False
)
ai_ouput = response.choices[0].message.content
print(ai_ouput)

{
    "response": "To sue your employer, you should first consult with a legal professional to discuss your specific situation and determine the best course of action. In Australia, the Fair Work Commission and various state-based anti-discrimination and workers' compensation tribunals handle employment disputes. Ensure you understand your rights under the Fair Work Act 2009 and any other relevant legislation. Gathering evidence and following the proper procedures for filing a claim is crucial.",
    "category": "legal"
}


In [10]:
try:
    json_format_ai_output = json.loads(ai_ouput)
    answer = json_format_ai_output.get('response', 'It was not possible to process that prompt')
    category = json_format_ai_output.get('category', 'general')
except json.JSONDecodeError:
    answer, category = ai_ouput, 'general'
    
DISCLAIMER_MESSAGE = (
    "\n\n**Disclaimer:** Be mindful that this is an AI assistant. "
    "Please consult with a professional before proceeding."
)
if category != 'general':
    answer += DISCLAIMER_MESSAGE

# update chat history
conversation.append(
    {
        'role' : 'assistant',
        'content' : answer
    }
)
for element in conversation:
    print(element)

{'role': 'system', 'content': 'You are a digital consultant specializing in Australia\'s evolving greenhouse gas (GHG) emission regulations.\nYour task is to help companies navigate the complexities of compliance, accurate emission calculations, and industry-specific scope definitions.\nYou also need to classify whether the user\'s question falls into the topics: [\'legal\', \'financial\', \'others\'] where \'others\' is any topic that is\nneither legal of financial.\nPlease only return JSON object the following structure:\n{\n    "response" : your answer to the user,\n    "category" : one of elements of [\'legal\', \'financial\', \'others\']\n}\n'}
{'role': 'user', 'content': 'how can i sue my employer?'}
{'role': 'assistant', 'content': "To sue your employer, you should first consult with a legal professional to discuss your specific situation and determine the best course of action. In Australia, the Fair Work Commission and various state-based anti-discrimination and workers' compe